In [1]:
import pandas as pd

from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""))

In [2]:
mf = pd.read_csv('mf_buildstock.csv', index_col='Building')

In [3]:
dirs = ['North', 'Northeast', 'East', 'Southeast', 'South', 'Southwest', 'West', 'Northwest']
inv_dirs = ['South', 'Southwest', 'West', 'Northwest', 'North', 'Northeast', 'East', 'Southeast']
orient_dict = dict(zip(dirs, inv_dirs))

In [4]:
master_df = pd.DataFrame()
for idx in mf.index:
    bldg = mf.loc[idx]
    units = int(bldg['Geometry Building Number Units MF'])
    floors = int(bldg['Geometry Building Floors'])
    num_units_per_floor = units/floors

    bstock = pd.DataFrame(columns= mf.columns, index=range(int(1),units+1))
    bstock.index.name = 'Building'
#     print("ID: ", idx)
#     print("floors: ", floors)
#     print("units: ", units)

    front_orient = bldg['Orientation']
    rear_orient = front_orient
    num_middle_z = floors - 2
    if num_units_per_floor%2 != 0: #no rear
        cor_val = 'Single Exterior Front'
        has_rear_units = False
        num_middle_x = int(num_units_per_floor - 2)
    else: #rear units
        cor_val = 'Double-Loaded Interior'
        has_rear_units = True
        rear_orient = orient_dict[bldg['Orientation']]
        num_middle_x = int((num_units_per_floor/2) - 2)   
    if num_middle_x < 0:
        num_middle_x = 0

    if has_rear_units:
        num_units_front = int(units/2)
        units_add = int(num_units_per_floor/2)
    else:
        num_units_front = int(units)
        units_add = int(num_units_per_floor)

    #Initial horizontal locations
    h_left = [1]
    if num_middle_x > 0:
        h_mid = list(range(2, 2+int(num_middle_x)))
    else:
        h_mid = []
    if num_units_front/floors > 1:
        h_right = [num_middle_x + 2]
    else:
        h_right = []
    #Define horizontal locatons by floor
    for f in range(2,floors+1):
        h_left += [h_left[-1] + units_add]
        h_mid += list(map(lambda x: x+units_add, h_mid[-num_middle_x:]))
        h_right += list(map(lambda x: x+units_add, h_right[-1:]))

    #LEVELS
    #..bottom
    if num_units_front/floors > 1:
        lv_bottom = list(range(1,2+num_middle_x+1))
    else:
        lv_bottom = [1]
    #..middle
    if num_middle_z > 0:
        lv_middle = list(range(lv_bottom[-1]+1, lv_bottom[-1]+1 + (units_add*num_middle_z)))
    else:
        lv_middle = []
    #..top
    if floors > 1:
        lv_top = list(range(units_add*(floors-1) + 1, units_add*floors +1))
    else:
        lv_top = []

    if has_rear_units:
        h_left += list(map(lambda x:x+num_units_front, h_left))
        h_mid += list(map(lambda x:x+num_units_front, h_mid))
        h_right += list(map(lambda x:x+num_units_front, h_right))
        lv_bottom += list(map(lambda x:x+num_units_front, lv_bottom))
        lv_middle += list(map(lambda x:x+num_units_front, lv_middle))
        lv_top += list(map(lambda x:x+num_units_front, lv_top))

    floor_unit = 1
    for unit in range(1,units+1):     
        bstock.loc[unit] = bldg
        bstock.loc[unit, 'Corridor'] = cor_val

        if unit in h_left:
            bstock.loc[unit, 'Geometry Single Unit Horizontal Location'] = 'Left'
        elif unit in h_mid:
            bstock.loc[unit, 'Geometry Single Unit Horizontal Location'] = 'Middle'
        elif unit in h_right:
            bstock.loc[unit, 'Geometry Single Unit Horizontal Location'] = 'Right'

        if unit in lv_bottom:
            bstock.loc[unit, 'Geometry Single Unit Level'] = 'Bottom'
        elif unit in lv_middle:
            bstock.loc[unit, 'Geometry Single Unit Level'] = 'Middle'
        elif unit in lv_top:
            bstock.loc[unit, 'Geometry Single Unit Level'] = 'Top'

        if unit <= units/2:
            bstock.loc[unit, 'Orientation'] = front_orient
        else:
            bstock.loc[unit, 'Orientation'] = rear_orient
            
#         bstock.loc[unit, 'Geometry Single Unit Has Rear Units'] = has_rear_units
    bstock['building_code'] = 'bldg_{}'.format(idx)
    if master_df.empty:
        master_df = bstock.copy()
    else:
        master_df = master_df.append(bstock)
#     bstock.to_csv('./mf_buildstock_singles/buildstock_' + str(idx) + '.csv')
master_df.index= range(1,len(master_df)+1)
master_df.index.rename('Building', inplace=True)

In [6]:
bldg_map = pd.DataFrame(columns=['building_range'], index=master_df.building_code.unique())
for bldg in master_df.building_code.unique():
    idx = list(master_df[master_df.building_code == bldg].index.values)
    bldg_map.loc[bldg]['building_range'] = '"{}"'.format(str(idx).replace(' ','').strip(']['))
bldg_map.head()
bldg_map.to_csv('C:/Users/aspeake/Documents/Projects/Load Profiles/Resstock_results/building_id_mapping.csv')

In [7]:
bldg_map

,building_range
bldg_1,"""1,2,3,4,5,6,7,8,9,10,11,12"""
bldg_2,"""13,14"""
bldg_3,"""15,16"""
bldg_4,"""17,18,19,20,21,22,23,24,25,26,27,28"""
bldg_5,"""29,30,31,32,33,34"""
bldg_6,"""35,36,37,38,39,40"""
bldg_7,"""41,42,43,44,45,46,47,48,49"""
bldg_8,"""50,51,52,53,54,55"""
bldg_9,"""56,57,58,59,60,61"""
bldg_10,"""62,63,64,65"""


In [87]:
master_df = master_df.drop('building_code', axis=1)
master_df.to_csv('buildstock_singleunits.csv')

In [ ]:
master_df.head()

In [6]:
bstock.filter(['Geometry Single Unit Level','Geometry Shared Walls SFA', 'Geometry Shared Walls MF','Geometry Single Unit Horizontal Location', 'Orientation'])

,Geometry Single Unit Level,Geometry Shared Walls SFA,Geometry Shared Walls MF,Geometry Single Unit Horizontal Location,Orientation
Building,,,,,
1,Bottom,None,None,Left,Northeast
2,Bottom,None,None,Middle,Northeast
3,Bottom,None,None,Right,Northeast
4,Middle,None,None,Left,Northeast
5,Middle,None,None,Middle,Northeast
6,Middle,None,None,Right,Northeast
7,Top,None,None,Left,Northeast
8,Top,None,None,Middle,Northeast
9,Top,None,None,Right,Northeast
